In [17]:
from io import open
from conllu import parse_incr

data_file = open("./ta_ttb-ud-train.conllu", "r", encoding="utf-8")
ud_files = []
for tokenlist in parse_incr(data_file):
    ud_files.append(tokenlist)

ud_treebank = []
for sentence in ud_files:
  tokens = []
  tags = []
  for token in sentence:
    tokens.append(token['form'])
    tags.append(token['upostag'])
  ud_treebank.append((tokens, tags))
print(ud_treebank[:5])

[(['சென்னை', 'அருகே', 'ஸ்ரீ', 'பெரும்புதூரில்', 'கிரீன்', 'பீல்டு', '(', 'நவீன', ')', 'விமான', 'நிலையத்துக்குக்கான', 'நிலையத்துக்குக்க்', 'ஆன', 'நிலம்', 'யாருக்கும்', 'பாதிப்பு', 'இல்லாத', 'வகையில்', 'எடுக்கப்', 'படும்', 'என்று', 'முதல்வர்', 'கருணாநிதி', 'உறுதியளித்துள்ளார்', 'உறுதியளித்த்', 'உள்ளார்', '.'], ['PROPN', 'ADP', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'ADJ', 'PUNCT', 'PROPN', '_', 'NOUN', 'PART', 'NOUN', 'PRON', 'NOUN', 'ADP', 'NOUN', 'VERB', 'AUX', 'PART', 'NOUN', 'PROPN', '_', 'VERB', 'AUX', 'PUNCT']), (['இது', 'தொடர்பாக', ',', 'அவர்', 'புதன்கிழமை', 'வெளியிட்ட', 'அறிக்கை', ':', '.'], ['PRON', 'ADV', 'PUNCT', 'PRON', 'NOUN', 'ADJ', 'NOUN', 'PUNCT', 'PUNCT']), (['நாடு', 'முழுவதும்', 'விமானப்', 'போக்குவரத்தில்', 'ஏற்பட்டு', 'வரும்', 'வளர்ச்சியைக்', 'கருத்தில்', 'கொண்டு', ',', 'முக்கிய', 'நகரங்களில்', 'உள்ள', 'விமான', 'நிலையங்களை', 'விரிவுபடுத்தவும்', 'விரிவுபடுத்தவ்', 'உம்', ',', 'புதிதாக', 'சர்வதேச', 'விமான', 'நிலையங்களை', 'அமைக்கவும்', 'அமைக்கவ்', 'உம்', 'மத்திய', 'அ

In [18]:
#Regex module for checking alphanumeric values.
import re
def extract_features(sentence, index):
  return {
      'word':sentence[index],
      'is_first':index==0,
      'is_last':index ==len(sentence)-1,
      'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
      'prefix-1':sentence[index][0],
      'prefix-2':sentence[index][:2],
      'prefix-3':sentence[index][:3],
      'prefix-4':sentence[index][:4],
      'suffix-1':sentence[index][-1],
      'suffix-2':sentence[index][-2:],
      'suffix-3':sentence[index][-3:],
      'suffix-4':sentence[index][-4:],
      'prev_word':'' if index == 0 else sentence[index-1],
      'next_word':'' if index < len(sentence) else sentence[index+1],
      'has_hyphen': '-' in sentence[index],
      'is_numeric': sentence[index].isdigit(),
  }

In [19]:

def transform_to_dataset(tagged_sentences):
  X, y = [], []
  for sentence, tags in tagged_sentences:
    sent_word_features, sent_tags = [],[]
    for index in range(len(sentence)):
        sent_word_features.append(extract_features(sentence, index)),
        sent_tags.append(tags[index])
    X.append(sent_word_features)
    y.append(sent_tags)
  return X, y

#UD Treebank.
ud_train_size = int(0.8*len(ud_treebank))
ud_training = ud_treebank[:ud_train_size]
ud_testing = ud_treebank[ud_train_size:]
X_ud_train, y_ud_train = transform_to_dataset(ud_training)
X_ud_test, y_ud_test = transform_to_dataset(ud_testing)


In [48]:
import warnings
warnings.filterwarnings('ignore')

from sklearn_crfsuite import CRF

ud_crf = CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=50,
    all_possible_transitions=True
)
print("Started training on UD corpus!")
ud_crf.fit(X_ud_train, y_ud_train)
print("Finished training on UD corpus!")

Started training on UD corpus!
Finished training on UD corpus!


In [49]:
#f1 score.
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers

print("## UD ##")

y_ud_pred=ud_crf.predict(X_ud_test)
print("F1 score on Test Data ")
print(metrics.flat_f1_score(y_ud_test, y_ud_pred,average='weighted',labels=ud_crf.classes_))
y_ud_pred_train=ud_crf.predict(X_ud_train)
print("F1 score on Training Data ")
print(metrics.flat_f1_score(y_ud_train, y_ud_pred_train,average='weighted',labels=ud_crf.classes_))


print("Class wise score:")
print(metrics.flat_classification_report(
    y_ud_test, y_ud_pred, labels=ud_crf.classes_, digits=3
))


## UD ##
F1 score on Test Data 
0.8491236392644625
F1 score on Training Data 
0.995707825887987
Class wise score:
              precision    recall  f1-score   support

       PROPN      0.877     0.757     0.813       169
         ADP      0.636     0.724     0.677        29
       PUNCT      1.000     1.000     1.000       117
         ADJ      0.876     0.821     0.848        95
           _      0.747     0.747     0.747        83
        NOUN      0.825     0.942     0.879       360
        PART      0.889     0.889     0.889        63
        PRON      0.952     0.606     0.741        33
        VERB      0.859     0.806     0.832       144
         AUX      0.838     0.870     0.854        77
         ADV      0.708     0.694     0.701        49
         DET      0.929     0.812     0.867        16
       CCONJ      1.000     1.000     1.000         4
         NUM      1.000     0.792     0.884        24

    accuracy                          0.850      1263
   macro avg      0.

In [22]:
sent = "பழுவேட்டரையர்! பழுவேட்டரையர்! – எங்கே, யாரிடம் பேசினாலும் பழுவேட்டரையர்களைப் பற்றியே பேச்சு!"
features = [extract_features(sent.split(), idx) for idx in range(len(sent.split()))]

ud_results = ud_crf.predict_single(features)

ud_tups = [(sent.split()[idx], ud_results[idx]) for idx in range(len(sent.split()))]

print(ud_tups)

[('பழுவேட்டரையர்!', 'PROPN'), ('பழுவேட்டரையர்!', 'PROPN'), ('–', 'PROPN'), ('எங்கே,', 'PROPN'), ('யாரிடம்', 'NOUN'), ('பேசினாலும்', '_'), ('பழுவேட்டரையர்களைப்', 'NOUN'), ('பற்றியே', 'ADP'), ('பேச்சு!', 'VERB')]


In [23]:
import pickle


penn_filename = 'penn_treebank_crf_postagger.sav'
pickle.dump(penn_crf, open(penn_filename, 'wb'))

ud_filename = 'ud_crf_postagger.sav'
pickle.dump(ud_crf, open(ud_filename,'wb'))